# Homework and bake-off: pragmatic color descriptions

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [All two-word examples as a dev corpus](#All-two-word-examples-as-a-dev-corpus)
1. [Dev dataset](#Dev-dataset)
1. [Random train–test split for development](#Random-train–test-split-for-development)
1. [Question 1: Improve the tokenizer [1 point]](#Question-1:-Improve-the-tokenizer-[1-point])
1. [Use the tokenizer](#Use-the-tokenizer)
1. [Question 2: Improve the color representations [1 point]](#Question-2:-Improve-the-color-representations-[1-point])
1. [Use the color representer](#Use-the-color-representer)
1. [Initial model](#Initial-model)
1. [Question 3: GloVe embeddings [1 points]](#Question-3:-GloVe-embeddings-[1-points])
1. [Try the GloVe representations](#Try-the-GloVe-representations)
1. [Question 4: Color context [3 points]](#Question-4:-Color-context-[3-points])
1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bakeoff [1 point]](#Bakeoff-[1-point])

## Overview

This homework and associated bake-off are oriented toward building an effective system for generating color descriptions that are pragmatic in the sense that they would help a reader/listener figure out which color was being referred to in a shared context consisting of a target color (whose identity is known only to the describer/speaker) and a set of distractors.

The notebook [colors_overview.ipynb](colors_overview.ipynb) should be studied before work on this homework begins. That notebook provides backgroud on the task, the dataset, and the modeling code that you will be using and adapting.

The homework questions are more open-ended than previous ones have been. Rather than asking you to implement pre-defined functionality, they ask you to try to improve baseline components of the full system in ways that you find to be effective. As usual, this culiminates in a prompt asking you to develop a novel system for entry into the bake-off. In this case, though, the work you do for the homework will likely be directly incorporated into that system.

## Set-up

See [colors_overview.ipynb](colors_overview.ipynb) for set-up in instructions and other background details.

In [1]:
from colors import ColorsCorpusReader
import os
from sklearn.model_selection import train_test_split
from torch_color_describer import (
    ColorizedNeuralListener, create_example_dataset)
import utils
from utils import START_SYMBOL, END_SYMBOL, UNK_SYMBOL
import numpy as np

In [2]:
utils.fix_random_seeds()

In [3]:
COLORS_SRC_FILENAME = os.path.join(
    "data", "colors", "filteredCorpus.csv")

## All two-word examples as a dev corpus

So that you don't have to sit through excessively long training runs during development, I suggest working with the two-word-only subset of the corpus until you enter into the late stages of system testing.

In [4]:
dev_corpus = ColorsCorpusReader(
    COLORS_SRC_FILENAME, 
    word_count=2, 
    normalize_colors=True)

In [6]:
dev_examples = list(dev_corpus.read())

This subset has about one-third the examples of the full corpus:

In [7]:
len(dev_examples)

13890

We __should__ worry that it's not a fully representative sample. Most of the descriptions in the full corpus are shorter, and a large proportion are longer. So this dataset is mainly for debugging, development, and general hill-climbing. All findings should be validated on the full dataset at some point.

## Dev dataset

The first step is to extract the raw color and raw texts from the corpus:

In [9]:
dev_rawcols, dev_texts = zip(*[[ex.colors, ex.contents] for ex in dev_examples])

The raw color representations are suitable inputs to a model, but the texts are just strings, so they can't really be processed as-is. Question 1 asks you to do some tokenizing!

## Random train–test split for development

For the sake of development runs, we create a random train–test split:

In [10]:
dev_rawcols_train, dev_rawcols_test, dev_texts_train, dev_texts_test = \
    train_test_split(dev_rawcols, dev_texts)

## Question 1: Improve the tokenizer [1 point]

This is the first required question – the first required modification to the default pipeline.

The function `tokenize_example` simply splits its string on whitespace and adds the required start and end symbols:

In [11]:
from colors_utils import heuristic_ending_tokenizer

def tokenize_example(s):
    
    # Improve me!
    
    return [START_SYMBOL] + heuristic_ending_tokenizer(s) + [END_SYMBOL]

def clean_test_and_training(dev_seqs_train, dev_seqs_test):    
    vocab = {}
    for toks in dev_seqs_train+dev_seqs_test:
        for w in toks:
            if w not in vocab:
                vocab[w]=0
            vocab[w]+=1
    removal_candidates = {k:v for k, v in vocab.items() if v == 1 }
    
    dev_seqs_train = [[w if w not in removal_candidates else UNK_SYMBOL for w in toks] for toks in dev_seqs_train]

    dev_seqs_test = [[w if w not in removal_candidates else UNK_SYMBOL for w in toks] for toks in dev_seqs_test]
    return dev_seqs_train, dev_seqs_test

In [12]:
tokenize_example(dev_texts_train[376])

['<s>', 'aqua', ',', 'teal', '</s>']

__Your task__: Modify `tokenize_example` so that it does something more sophisticated with the input text. 

__Notes__:

* There are useful ideas for this in [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142)
* There is no requirement that you do word-level tokenization. Sub-word and multi-word are options.
* This question can interact with the size of your vocabulary (see just below), and in turn with decisions about how to use `UNK_SYMBOL`.

__Important__: don't forget to add the start and end symbols, else the resulting models will definitely be terrible!

## Use the tokenizer

Once the tokenizer is working, run the following cell to tokenize your inputs:

In [18]:
dev_seqs_train = [tokenize_example(s) for s in dev_texts_train]

dev_seqs_test = [tokenize_example(s) for s in dev_texts_test]

dev_seqs_train, dev_seqs_test = clean_test_and_training(dev_seqs_train, dev_seqs_test)

We use only the train set to derive a vocabulary for the model:

In [19]:
dev_vocab = sorted({w for toks in dev_seqs_train for w in toks}) + [UNK_SYMBOL]

It's important that the `UNK_SYMBOL` is included somewhere in this list. Test examples with word not seen in training will be mapped to `UNK_SYMBOL`. If you model's vocab is the same as your train vocab, then `UNK_SYMBOL` will never be encountered during training, so it will be a random vector at test time.

In [22]:
len(dev_vocab)

524

## Question 2: Improve the color representations [1 point]

This is the second required pipeline improvement for the assignment. 

The following functions do nothing at all to the raw input colors we get from the corpus. 

In [23]:
import colorsys

def represent_color_context(colors):
    
    # Improve me!
    
    return [represent_color(color) for color in colors]


def represent_color(color):
    #import numpy.fft as fft
    # Improve me!
    #return color
    return colorsys.rgb_to_hsv(*color)

In [24]:
represent_color_context(dev_rawcols_train[0])

[(0.2972459639126306, 0.78, 0.5),
 (0.08032596041909196, 0.6386617100371748, 0.7472222222222222),
 (0.5837378640776699, 0.6270928462709285, 0.73)]

__Your task__: Modify `represent_color_context` and/or `represent_color` to represent colors in a new way.
    
__Notes__:

* The Fourier-transform method of [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142) is a proven choice.
* You are not required to keep `represent_color`. This might be unnatural if you want to perform an operation on each color trio all at once.
* For that matter, if you want to process all of the color contexts in the entire data set all at once, that is fine too, as long as you can also perform the operation at test time with an unknown number of examples being tested.

## Use the color representer

The following cell just runs your `represent_color_context` on the train and test sets:

In [25]:
dev_cols_train = [represent_color_context(colors) for colors in dev_rawcols_train]

dev_cols_test = [represent_color_context(colors) for colors in dev_rawcols_test]

At this point, our preprocessing steps are complete, and we can fit a first model.

## Initial model

The first model is configured right now to be a small model run for just a few iterations. It should be enough to get traction, but it's unlikely to be a great model. You are free to modify this configuration if you wish; it is here just for demonstration and testing:

In [26]:
dev_mod = ColorizedNeuralListener(
    dev_vocab, 
    embed_dim=10, 
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)

Using cpu


In [29]:
_ = dev_mod.fit(dev_cols_train, dev_seqs_train)

ColorizedNeuralListenerEncoder cpu
ColorizedNeuralListenerEncoderDecoder cpu
Train: Epoch 1; err = 84.04119199514389; time = 1.5688772201538086
Train: Epoch 2; err = 77.22541165351868; time = 1.4339301586151123
Train: Epoch 3; err = 75.3536547422409; time = 1.4345111846923828
Train: Epoch 4; err = 73.96716660261154; time = 1.4857251644134521
Train: Epoch 5; err = 73.30008709430695; time = 1.533970832824707


We can also see the model's predicted sequences given color context inputs:

In [30]:
dev_mod.predict(dev_cols_test[:1], dev_seqs_train[:1])

[2]

As discussed in [colors_overview.ipynb](colors_overview.ipynb), our primary metric is `listener_accuracy`:

In [32]:
#dev_mod.listener_accuracy(dev_cols_test, dev_seqs_test)

In [22]:
#dev_seqs_train[:1]

## Question 3: GloVe embeddings [1 points]

The above model uses a random initial embedding, as configured by the decoder used by `ContextualColorDescriber`. This homework question asks you to consider using GloVe inputs. 

__Your task__: Complete `create_glove_embedding` so that it creates a GloVe embedding based on your model vocabulary. This isn't mean to be analytically challenging, but rather just to create a basis for you to try out other kinds of rich initialization.

In [33]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [34]:
def create_glove_embedding(vocab, glove_base_filename='glove.6B.100d.txt'):
    
    # Use `utils.glove2dict` to read in the GloVe file:    
    ##### YOUR CODE HERE
    glove_dict = utils.glove2dict(os.path.join(GLOVE_HOME, glove_base_filename))

    
    # Use `utils.create_pretrained_embedding` to create the embedding.
    # This function will, by default, ensure that START_TOKEN, 
    # END_TOKEN, and UNK_TOKEN are included in the embedding.
    ##### YOUR CODE HERE
    embedding, new_vocab = utils.create_pretrained_embedding(glove_dict, vocab)

    
    # Be sure to return the embedding you create as well as the
    # vocabulary returned by `utils.create_pretrained_embedding`,
    # which is likely to have been modified from the input `vocab`.
    
    ##### YOUR CODE HERE
    return embedding, new_vocab


## Try the GloVe representations

Let's see if GloVe helped for our development data:

In [25]:
#dev_glove_embedding, dev_glove_vocab = create_glove_embedding(dev_vocab)

In [35]:
embedding = np.random.normal(
            loc=0, scale=0.01, size=(len(dev_vocab), 100))

The above might dramatically change your vocabulary, depending on how many items from your vocab are in the Glove space:

## Question 4: Color context [3 points]

In [36]:
toy_color_seqs, toy_word_seqs, toy_vocab = create_example_dataset(
    group_size=50, vec_dim=2)

In [37]:
toy_color_seqs_train, toy_color_seqs_test, toy_word_seqs_train, toy_word_seqs_test = \
    train_test_split(toy_color_seqs, toy_word_seqs)

In [38]:
toy_mod = ColorizedNeuralListener(
    toy_vocab, 
    embed_dim=100, 
    embedding=embedding,
    hidden_dim=100, 
    max_iter=100, 
    batch_size=128)

Using cpu


In [39]:
_ = toy_mod.fit(toy_color_seqs_train, toy_word_seqs_train)

ColorizedNeuralListenerEncoder cpu
ColorizedNeuralListenerEncoderDecoder cpu
Train: Epoch 1; err = 1.0981638431549072; time = 0.02631402015686035
Train: Epoch 2; err = 1.0931386947631836; time = 0.01901078224182129
Train: Epoch 3; err = 1.0778679847717285; time = 0.0189511775970459
Train: Epoch 4; err = 1.041054129600525; time = 0.018870830535888672
Train: Epoch 5; err = 1.0372779369354248; time = 0.0178070068359375
Train: Epoch 6; err = 0.9852323532104492; time = 0.015791893005371094
Train: Epoch 7; err = 0.9883937239646912; time = 0.01615595817565918
Train: Epoch 8; err = 0.9423364996910095; time = 0.014310121536254883
Train: Epoch 9; err = 0.9208185076713562; time = 0.014590024948120117
Train: Epoch 10; err = 0.9072319865226746; time = 0.013576030731201172
Train: Epoch 11; err = 0.8765920400619507; time = 0.014236211776733398
Train: Epoch 12; err = 0.8645085692405701; time = 0.014837026596069336
Train: Epoch 13; err = 0.87237948179245; time = 0.015115022659301758
Train: Epoch 14; er

In [40]:
preds = toy_mod.predict(toy_color_seqs_test, toy_word_seqs_test)
correct = sum([1 if x == 2 else 0 for x in preds])
print(correct, "/", len(preds), correct/len(preds))

26 / 38 0.6842105263157895


If that worked, then you can now try this model on SCC problems!

In [41]:
dev_color_mod = ColorizedNeuralListener(
    dev_vocab, 
    #embedding=dev_glove_embedding, 
    embed_dim=100,
    embedding=embedding,
    hidden_dim=100, 
    max_iter=500,
    batch_size=64,
    dropout_prob=0.,
    eta=0.005,
    lr_rate=0.96,
    warm_start=True,
    force_cpu=False)

Using cpu


In [42]:
_ = dev_color_mod.fit(dev_cols_train, dev_seqs_train)

ColorizedNeuralListenerEncoder cpu
ColorizedNeuralListenerEncoderDecoder cpu
Train: Epoch 1; err = 172.73405945301056; time = 2.005457878112793
Train: Epoch 2; err = 164.6625559926033; time = 2.011399984359741
Train: Epoch 3; err = 159.40583962202072; time = 2.0694618225097656
Train: Epoch 4; err = 156.94196170568466; time = 2.1623833179473877
Train: Epoch 5; err = 153.71348571777344; time = 2.159597158432007
Train: Epoch 6; err = 151.77801221609116; time = 2.248502016067505
Train: Epoch 7; err = 151.15116107463837; time = 2.187623977661133
Train: Epoch 8; err = 150.12537652254105; time = 2.2472281455993652
Train: Epoch 9; err = 148.91111654043198; time = 2.2541627883911133
Train: Epoch 10; err = 147.8804365992546; time = 2.2705888748168945
Train: Epoch 11; err = 147.38216990232468; time = 2.227753162384033
Train: Epoch 12; err = 147.5932126045227; time = 2.412477970123291
Train: Epoch 13; err = 146.85077250003815; time = 2.2686657905578613
Train: Epoch 14; err = 145.15704756975174; ti

Train: Epoch 110; err = 122.83343231678009; time = 2.8297810554504395
Train: Epoch 111; err = 123.14543253183365; time = 2.848889112472534
Train: Epoch 112; err = 123.55892890691757; time = 2.9393112659454346
Train: Epoch 113; err = 122.89370036125183; time = 2.8181040287017822
Train: Epoch 114; err = 122.61023271083832; time = 2.7814948558807373
Train: Epoch 115; err = 122.52508795261383; time = 2.8210718631744385
Train: Epoch 116; err = 122.1898283958435; time = 2.785043954849243
Train: Epoch 117; err = 122.06646406650543; time = 2.6946470737457275
Train: Epoch 118; err = 122.64717185497284; time = 2.776918649673462
Train: Epoch 119; err = 122.15429109334946; time = 2.6055917739868164
0.003606947894919167
tensor([0.0900, 0.0608, 0.8493], grad_fn=<MeanBackward1>) 0.6857959032058716
Train: Epoch 120; err = 122.32560449838638; time = 2.4888927936553955
Train: Epoch 121; err = 122.82581549882889; time = 2.548144817352295
Train: Epoch 122; err = 121.4633594751358; time = 2.581407070159912

Train: Epoch 219; err = 117.8572399020195; time = 2.7460718154907227
Train: Epoch 220; err = 117.59159523248672; time = 2.632805109024048
Train: Epoch 221; err = 118.17557549476624; time = 2.6236510276794434
Train: Epoch 222; err = 117.3917595744133; time = 2.5967681407928467
Train: Epoch 223; err = 117.64069038629532; time = 2.630458116531372
Train: Epoch 224; err = 117.45365226268768; time = 2.551806926727295
0.0027104318993045437
tensor([0.0693, 0.1437, 0.7870], grad_fn=<MeanBackward1>) 0.746882438659668
Train: Epoch 225; err = 117.33862388134003; time = 2.54649019241333
Train: Epoch 226; err = 117.33894658088684; time = 2.519274950027466
Train: Epoch 227; err = 117.19848906993866; time = 2.490694999694824
Train: Epoch 228; err = 116.92347627878189; time = 2.432830810546875
Train: Epoch 229; err = 117.42016166448593; time = 2.4875340461730957
Train: Epoch 230; err = 117.13965398073196; time = 2.486039161682129
Train: Epoch 231; err = 117.7436056137085; time = 2.4469308853149414
Trai

Train: Epoch 328; err = 115.42594057321548; time = 3.2626218795776367
Train: Epoch 329; err = 115.24288266897202; time = 3.181856155395508
0.0020367472153163093
tensor([0.1543, 0.1064, 0.7393], grad_fn=<MeanBackward1>) 0.7997725605964661
Train: Epoch 330; err = 115.53066271543503; time = 3.1442790031433105
Train: Epoch 331; err = 115.10750848054886; time = 3.130434989929199
Train: Epoch 332; err = 115.2725841999054; time = 3.0072782039642334
Train: Epoch 333; err = 115.04211854934692; time = 3.0877339839935303
Train: Epoch 334; err = 115.21911191940308; time = 3.0117359161376953
Train: Epoch 335; err = 115.6122055053711; time = 2.909853935241699
Train: Epoch 336; err = 115.32840079069138; time = 2.903993844985962
Train: Epoch 337; err = 116.18538790941238; time = 2.990070104598999
Train: Epoch 338; err = 115.77827286720276; time = 2.9947142601013184
Train: Epoch 339; err = 115.67922914028168; time = 3.1191461086273193
Train: Epoch 340; err = 115.27143037319183; time = 3.223014116287231

Train: Epoch 436; err = 114.09270018339157; time = 2.5489258766174316
Train: Epoch 437; err = 114.2434458732605; time = 2.588402032852173
Train: Epoch 438; err = 114.07116204500198; time = 2.580483913421631
Train: Epoch 439; err = 114.12103819847107; time = 2.5501868724823
Train: Epoch 440; err = 114.12211203575134; time = 2.570964813232422
Train: Epoch 441; err = 114.15007942914963; time = 2.563708782196045
Train: Epoch 442; err = 114.03636240959167; time = 2.592534065246582
Train: Epoch 443; err = 114.05013114213943; time = 2.643465995788574
Train: Epoch 444; err = 113.90775513648987; time = 2.6327271461486816
Train: Epoch 445; err = 114.3116842508316; time = 2.6236751079559326
Train: Epoch 446; err = 114.86062318086624; time = 2.7655909061431885
Train: Epoch 447; err = 114.53568398952484; time = 2.9142239093780518
Train: Epoch 448; err = 114.62177670001984; time = 3.0528581142425537
Train: Epoch 449; err = 113.87623065710068; time = 3.0456759929656982
0.0014692882161535274
tensor([0

In [43]:
test_preds = dev_color_mod.predict(dev_cols_test, dev_seqs_test)
#dev_color_mod.predict(dev_cols_test, dev_seqs_test, probabilities=True)
train_preds = dev_color_mod.predict(dev_cols_train, dev_seqs_train)
#dev_color_mod.predict(dev_cols_test, dev_seqs_test, probabilities=True)

In [45]:
correct = sum([1 if x == 2 else 0 for x in test_preds])
print("test", correct, "/", len(test_preds), correct/len(test_preds))
correct = sum([1 if x == 2 else 0 for x in train_preds])
print("train", correct, "/", len(train_preds), correct/len(train_preds))

test 2437 / 3473 0.70169881946444
train 8962 / 10417 0.8603244696169723


In [52]:
hyperparameter_exploration = {
            "batch_medium": {
                "batch_size": 64,
                "dropout_prob": 0.0,
                "eta": 0.005,
                "lr_rate": 0.96,
            },
            "batch_small": {
                "batch_size": 32,
                "dropout_prob": 0.0,
                "eta": 0.005,
                "lr_rate": 0.96,
            },
             "batch_xsmall": {
                "batch_size": 16,
                "dropout_prob": 0.0,
                "eta": 0.005,
                "lr_rate": 0.96,
            },
            "batch_large": {
                "batch_size": 128,
                "dropout_prob": 0.0,
                "eta": 0.005,
                "lr_rate": 0.96,
            },
            "batch_xlarge": {
                "batch_size": 256,
                "dropout_prob": 0.0,
                "eta": 0.005,
                "lr_rate": 0.96,
            },
            "dropout_small": {
                "batch_size": 64,
                "dropout_prob": 0.1,
                "eta": 0.005,
                "lr_rate": 0.96,
            },
            "dropout_large": {
                "batch_size": 64,
                "dropout_prob": 0.3,
                "eta": 0.005,
                "lr_rate": 0.96,
            },
            "eta_small": {
                "batch_size": 64,
                "dropout_prob": 0.0,
                "eta": 0.0005,
                "lr_rate": 0.96,
            },
            "eta_medium": {
                "batch_size": 64,
                "dropout_prob": 0.0,
                "eta": 0.01,
                "lr_rate": 0.96,
            },
            "eta_large": {
                "batch_size": 64,
                "dropout_prob": 0.0,
                "eta": 0.05,
                "lr_rate": 0.96,
            },
            "lr_small": {
                "batch_size": 64,
                "dropout_prob": 0.0,
                "eta": 0.005,
                "lr_rate": 0.05,
            },
            "lr_medium": {
                "batch_size": 64,
                "dropout_prob": 0.0,
                "eta": 0.005,
                "lr_rate": 0.4,
            },
            "lr_large": {
                "batch_size": 64,
                "dropout_prob": 0.0,
                "eta": 0.005,
                "lr_rate": 0.99,
            },
        }

model_exploration = {}
model_fittings = {}
for config in hyperparameter_exploration:
    print(f'\nTraining for config: {config}')
    model_exploration[config] = ColorizedNeuralListener(
        dev_vocab, 
        embed_dim=100,
        embedding=embedding,
        hidden_dim=100, 
        max_iter=400,
        warm_start=True,
        force_cpu=False, **hyperparameter_exploration[config])
    model_fittings[config] = model_exploration[config].fit(dev_cols_train, dev_seqs_train)


Training for config: batch_medium
Using cpu
ColorizedNeuralListenerEncoder cpu
ColorizedNeuralListenerEncoderDecoder cpu
Train: Epoch 1; err = 172.4085202217102; time = 1.9026310443878174
Train: Epoch 2; err = 163.1066757440567; time = 1.9381968975067139
Train: Epoch 3; err = 159.3544823527336; time = 1.8862369060516357
Train: Epoch 4; err = 156.59143388271332; time = 1.910494089126587
Train: Epoch 5; err = 154.4408074617386; time = 2.0201563835144043
Train: Epoch 6; err = 153.5412660241127; time = 2.045928955078125
Train: Epoch 7; err = 152.05450975894928; time = 2.0667338371276855
Train: Epoch 8; err = 151.10628414154053; time = 2.019474983215332
Train: Epoch 9; err = 150.33194142580032; time = 2.0745432376861572
Train: Epoch 10; err = 149.6664039492607; time = 2.1198348999023438
Train: Epoch 11; err = 150.09579610824585; time = 2.1080482006073
Train: Epoch 12; err = 149.47756612300873; time = 2.155155897140503
Train: Epoch 13; err = 147.82903039455414; time = 2.1725730895996094
Tra

Train: Epoch 110; err = 124.1750248670578; time = 2.770617961883545
Train: Epoch 111; err = 123.77630805969238; time = 2.8097243309020996
Train: Epoch 112; err = 124.31997841596603; time = 2.8579342365264893
Train: Epoch 113; err = 124.07885468006134; time = 2.984311819076538
Train: Epoch 114; err = 124.22640067338943; time = 2.9559879302978516
Train: Epoch 115; err = 124.45721304416656; time = 2.9508988857269287
Train: Epoch 116; err = 123.63563448190689; time = 2.8443820476531982
Train: Epoch 117; err = 122.85251623392105; time = 2.6955888271331787
Train: Epoch 118; err = 123.96957570314407; time = 2.5170552730560303
Train: Epoch 119; err = 123.94109219312668; time = 2.4478278160095215
0.003606947894919167
tensor([0.1234, 0.1469, 0.7297], grad_fn=<MeanBackward1>) 0.8117360472679138
Train: Epoch 120; err = 123.46891552209854; time = 2.358455181121826
Train: Epoch 121; err = 122.72969734668732; time = 2.3089840412139893
Train: Epoch 122; err = 123.51391178369522; time = 2.2747161388397

Train: Epoch 219; err = 119.80273312330246; time = 2.4867289066314697
Train: Epoch 220; err = 119.26295530796051; time = 2.4214260578155518
Train: Epoch 221; err = 119.87914246320724; time = 2.4540328979492188
Train: Epoch 222; err = 120.74695187807083; time = 2.363023281097412
Train: Epoch 223; err = 119.94810795783997; time = 2.3910257816314697
Train: Epoch 224; err = 119.91062599420547; time = 2.322575092315674
0.0027104318993045437
tensor([0.1132, 0.0853, 0.8015], grad_fn=<MeanBackward1>) 0.7346795797348022
Train: Epoch 225; err = 119.6625167131424; time = 2.327266216278076
Train: Epoch 226; err = 119.35956281423569; time = 2.312695026397705
Train: Epoch 227; err = 118.94033843278885; time = 2.2695250511169434
Train: Epoch 228; err = 118.6839325428009; time = 2.2539830207824707
Train: Epoch 229; err = 118.97984826564789; time = 2.2742412090301514
Train: Epoch 230; err = 118.8381689786911; time = 2.31502103805542
Train: Epoch 231; err = 119.01726430654526; time = 2.2976088523864746


Train: Epoch 328; err = 117.1567901968956; time = 2.3762741088867188
Train: Epoch 329; err = 116.9958900809288; time = 2.3632149696350098
0.0020367472153163093
tensor([0.1284, 0.0486, 0.8230], grad_fn=<MeanBackward1>) 0.7169910073280334
Train: Epoch 330; err = 117.15183299779892; time = 2.3852882385253906
Train: Epoch 331; err = 116.96791714429855; time = 2.3623158931732178
Train: Epoch 332; err = 117.26266252994537; time = 2.3805341720581055
Train: Epoch 333; err = 116.92461436986923; time = 2.364043712615967
Train: Epoch 334; err = 117.20121556520462; time = 2.3708059787750244
Train: Epoch 335; err = 117.37174379825592; time = 2.444483995437622
Train: Epoch 336; err = 116.98427188396454; time = 2.4567487239837646
Train: Epoch 337; err = 117.26569133996964; time = 2.466336965560913
Train: Epoch 338; err = 117.00214296579361; time = 2.5048630237579346
Train: Epoch 339; err = 117.70784336328506; time = 2.5522191524505615
Train: Epoch 340; err = 117.52807360887527; time = 2.5126247406005

Train: Epoch 36; err = 266.4420014619827; time = 2.988229751586914
Train: Epoch 37; err = 266.4993517398834; time = 2.8967559337615967
Train: Epoch 38; err = 264.99391639232635; time = 2.9232068061828613
Train: Epoch 39; err = 266.83353340625763; time = 2.9137189388275146
Train: Epoch 40; err = 271.2175948023796; time = 2.9002459049224854
Train: Epoch 41; err = 262.7966698408127; time = 2.917358875274658
Train: Epoch 42; err = 263.34832590818405; time = 2.8705012798309326
Train: Epoch 43; err = 264.0669021010399; time = 2.9269349575042725
Train: Epoch 44; err = 263.39954620599747; time = 2.904902935028076
0.004423679999999999
tensor([0.1006, 0.0822, 0.8172], grad_fn=<MeanBackward1>) 0.7098647952079773
Train: Epoch 45; err = 260.7729706168175; time = 3.002161979675293
Train: Epoch 46; err = 260.3976284265518; time = 3.0469791889190674
Train: Epoch 47; err = 260.45979714393616; time = 3.115813970565796
Train: Epoch 48; err = 260.04351633787155; time = 3.1853020191192627
Train: Epoch 49; 

...

Train: Epoch 286; err = 129.24376326799393; time = 2.170297861099243
Train: Epoch 287; err = 129.2612493634224; time = 2.1540610790252686
Train: Epoch 288; err = 129.24444091320038; time = 2.1940441131591797
Train: Epoch 289; err = 129.24168121814728; time = 2.1950840950012207
Train: Epoch 290; err = 129.25026762485504; time = 2.2105119228363037
Train: Epoch 291; err = 129.24892270565033; time = 2.1729800701141357
Train: Epoch 292; err = 129.2515977025032; time = 2.199542999267578
Train: Epoch 293; err = 129.25606137514114; time = 2.219412088394165
Train: Epoch 294; err = 129.25438559055328; time = 2.1623270511627197
Train: Epoch 295; err = 129.22810804843903; time = 2.193969249725342
Train: Epoch 296; err = 129.2596788406372; time = 2.1721081733703613
Train: Epoch 297; err = 129.24147087335587; time = 2.169823169708252
Train: Epoch 298; err = 129.25138294696808; time = 2.1699719429016113
Train: Epoch 299; err = 129.228131711483; time = 2.14973521232605
5.497558138880007e-11
tensor([0.

Train: Epoch 395; err = 129.22668832540512; time = 2.151151180267334
Train: Epoch 396; err = 129.25816023349762; time = 2.1639890670776367
Train: Epoch 397; err = 129.25356394052505; time = 2.1721198558807373
Train: Epoch 398; err = 129.2359681725502; time = 2.152987241744995
Train: Epoch 399; err = 129.23490422964096; time = 2.2081918716430664
Train: Epoch 400; err = 129.2484648823738; time = 2.1798830032348633

Training for config: lr_large
Using cpu
ColorizedNeuralListenerEncoder cpu
ColorizedNeuralListenerEncoderDecoder cpu
Train: Epoch 1; err = 176.57816982269287; time = 2.1914620399475098
Train: Epoch 2; err = 165.9784476161003; time = 2.220134973526001
Train: Epoch 3; err = 160.83957117795944; time = 2.174172878265381
Train: Epoch 4; err = 158.21571403741837; time = 2.1849608421325684
Train: Epoch 5; err = 155.59450215101242; time = 2.210139274597168
Train: Epoch 6; err = 153.746131837368; time = 2.1834776401519775
Train: Epoch 7; err = 152.86804401874542; time = 2.2215700149536

Train: Epoch 106; err = 127.36018490791321; time = 2.1582720279693604
Train: Epoch 107; err = 127.70705789327621; time = 2.169701099395752
Train: Epoch 108; err = 125.78749519586563; time = 2.2320430278778076
Train: Epoch 109; err = 126.50278323888779; time = 2.220720052719116
Train: Epoch 110; err = 126.02510166168213; time = 2.1894540786743164
Train: Epoch 111; err = 125.8369511961937; time = 2.159778118133545
Train: Epoch 112; err = 125.29536157846451; time = 2.2101712226867676
Train: Epoch 113; err = 126.76042973995209; time = 2.24767804145813
Train: Epoch 114; err = 125.88960874080658; time = 2.1537699699401855
Train: Epoch 115; err = 125.06680399179459; time = 2.258060932159424
Train: Epoch 116; err = 126.2240480184555; time = 2.202198028564453
Train: Epoch 117; err = 125.50231283903122; time = 2.231013059616089
Train: Epoch 118; err = 125.42477136850357; time = 2.270094871520996
Train: Epoch 119; err = 124.59617614746094; time = 2.259223699569702
0.0046137234721396
tensor([0.106

Train: Epoch 215; err = 118.99971926212311; time = 2.2440390586853027
Train: Epoch 216; err = 119.18001371622086; time = 2.215074062347412
Train: Epoch 217; err = 120.37658071517944; time = 2.2680959701538086
Train: Epoch 218; err = 120.52789342403412; time = 2.2233500480651855
Train: Epoch 219; err = 119.53441953659058; time = 2.23516583442688
Train: Epoch 220; err = 119.00841856002808; time = 2.2215330600738525
Train: Epoch 221; err = 118.80336767435074; time = 2.257906198501587
Train: Epoch 222; err = 119.3019151687622; time = 2.245568037033081
Train: Epoch 223; err = 119.09828305244446; time = 2.2609200477600098
Train: Epoch 224; err = 118.93870228528976; time = 2.262495994567871
0.004300291773206443
tensor([0.1267, 0.0344, 0.8389], grad_fn=<MeanBackward1>) 0.6998190879821777
Train: Epoch 225; err = 118.94570994377136; time = 2.295281171798706
Train: Epoch 226; err = 119.46868258714676; time = 2.2526116371154785
Train: Epoch 227; err = 120.11539924144745; time = 2.2507503032684326


Train: Epoch 324; err = 117.91102516651154; time = 2.345062017440796
Train: Epoch 325; err = 117.42412900924683; time = 2.299931049346924
Train: Epoch 326; err = 117.34259700775146; time = 2.308371067047119
Train: Epoch 327; err = 116.91366821527481; time = 2.31630277633667
Train: Epoch 328; err = 116.78759241104126; time = 2.2712080478668213
Train: Epoch 329; err = 116.87813329696655; time = 2.3009021282196045
0.00400815294769523
tensor([0.1051, 0.1093, 0.7856], grad_fn=<MeanBackward1>) 0.7515583634376526
Train: Epoch 330; err = 116.95521938800812; time = 2.2894909381866455
Train: Epoch 331; err = 116.82769793272018; time = 2.2857301235198975
Train: Epoch 332; err = 116.5975952744484; time = 2.256218910217285
Train: Epoch 333; err = 117.0042929649353; time = 2.2956011295318604
Train: Epoch 334; err = 118.07951009273529; time = 2.311537981033325
Train: Epoch 335; err = 117.25062131881714; time = 2.290397882461548
Train: Epoch 336; err = 116.77962332963943; time = 2.3208131790161133
Tra

In [53]:
for config, model in model_exploration.items():
    print(f'\nResults for {config}')
    test_preds = model.predict(dev_cols_test, dev_seqs_test)
    #dev_color_mod.predict(dev_cols_test, dev_seqs_test, probabilities=True)
    train_preds = model.predict(dev_cols_train, dev_seqs_train)
    #dev_color_mod.predict(dev_cols_test, dev_seqs_test, probabilities=True)
    correct = sum([1 if x == 2 else 0 for x in test_preds])
    print("test", correct, "/", len(test_preds), correct/len(test_preds))
    correct = sum([1 if x == 2 else 0 for x in train_preds])
    print("train", correct, "/", len(train_preds), correct/len(train_preds))


Results for batch_medium
test 2385 / 3473 0.6867261733371725
train 8806 / 10417 0.8453489488336373

Results for batch_small
test 2409 / 3473 0.6936366253959113
train 8928 / 10417 0.85706057406163

Results for batch_xsmall
test 2417 / 3473 0.6959401094154909
train 8959 / 10417 0.8600364788326773

Results for batch_large
test 2422 / 3473 0.6973797869277282
train 8818 / 10417 0.8465009119708169

Results for batch_xlarge
test 2399 / 3473 0.6907572703714367
train 8701 / 10417 0.8352692713833157

Results for dropout_small
test 2214 / 3473 0.6374892024186583
train 8330 / 10417 0.7996544110588462

Results for dropout_large
test 2175 / 3473 0.6262597178232076
train 8066 / 10417 0.7743112220408946

Results for eta_small
test 2268 / 3473 0.6530377195508206
train 7526 / 10417 0.7224728808678123

Results for eta_medium
test 2407 / 3473 0.6930607543910164
train 8935 / 10417 0.8577325525583182

Results for eta_large
test 2282 / 3473 0.657068816585085
train 7583 / 10417 0.7279447057694154

Results fo

In [41]:
totals = {}
for ex in dev_examples:
    #ex.display(typ='speaker')
    #print(ex.condition)
    if ex.condition not in totals:
        totals[ex.condition] = 0
    totals[ex.condition]+=1
    #print(dev_color_mod.predict([ex.speaker_context], [tokenize_example(ex.contents)], probabilities=True))
    #print(dev_color_mod.predict([ex.speaker_context], [tokenize_example(ex.contents)])[0])
    #print()
    
scores = {}
for ex in dev_examples:
    #ex.display(typ='speaker')
    #print(ex.condition)
    if ex.condition not in scores:
        scores[ex.condition] = 0
    if dev_color_mod.predict([represent_color_context(ex.colors)], [tokenize_example(ex.contents)])[0] == 2:
        scores[ex.condition]+=1

In [42]:
for condition in scores:
    print(condition, ":", scores[condition], "/", totals[condition], "=", scores[condition]/totals[condition])

close : 4071 / 5776 = 0.7048130193905817
far : 2009 / 2657 = 0.756115920210764
split : 3869 / 5457 = 0.7089976177386843


In [43]:
#dev_perp = dev_color_mod.perplexities(dev_cols_test, dev_seqs_test)
#dev_perp[0]

In [44]:
#dev_color_mod.to_pickle(os.path.join('data', 'colors' 'color_describer_unigram_20e.pt'))